In [224]:
import pandas as pd
import os

pagos_por_cat_hand = pd.read_pickle(r'C:\Users\emita\Bootcamp\EDA\PICKLES\pagos_hand_limpio.pkl')
pagos_por_cat_hand.info()

<class 'pandas.core.frame.DataFrame'>
Index: 210 entries, 0 to 42
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CATEGORIA  210 non-null    object
 1   APELLIDO   210 non-null    object
 2   NOMBRE     210 non-null    object
 3   SEGURO     210 non-null    object
 4   FEB        210 non-null    object
 5   MAR        210 non-null    object
 6   ABR        210 non-null    object
 7   MAY        210 non-null    object
 8   JUN        210 non-null    object
 9   JUL        210 non-null    object
 10  AGO        210 non-null    object
 11  SEPT       210 non-null    object
 12  OCT        210 non-null    object
 13  DEUDA      210 non-null    int64 
dtypes: int64(1), object(13)
memory usage: 24.6+ KB


In [225]:
pagos_por_cat_voley = pd.read_pickle(r'C:\Users\emita\Bootcamp\EDA\PICKLES\pagos_voley_limpio.pkl')
pagos_por_cat_voley.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240 entries, 0 to 27
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CATEGORIA  240 non-null    object 
 1   APELLIDO   240 non-null    object 
 2   NOMBRE     240 non-null    object 
 3   SEGURO     240 non-null    object 
 4   FEB        240 non-null    object 
 5   MAR        240 non-null    object 
 6   ABR        240 non-null    object 
 7   MAY        240 non-null    object 
 8   JUN        240 non-null    object 
 9   JUL        240 non-null    object 
 10  AGO        240 non-null    object 
 11  SEPT       240 non-null    object 
 12  OCT        240 non-null    object 
 13  DEUDA      240 non-null    float64
dtypes: float64(1), object(13)
memory usage: 28.1+ KB


## Socios: Ingresos, egresos, permanencias y porcentajes

In [226]:
def ingresos_egresos_permanencias(df):
    # Crear un DataFrame vacío para almacenar los resultados
    resultados = pd.DataFrame(columns=['CATEGORIA', 'MES', 'INGRESOS', 'EGRESOS', 'PERMANENCIAS'])
    
    # Definir los valores que representan presencia
    presente = ["Lesion", "X", "Beca"]
    
    # Definir las columnas de los meses
    meses = ['FEB', 'MAR', 'ABR', 'MAY', 'JUN', 'JUL', 'AGO', 'SEPT', 'OCT']
    
    # Iterar sobre cada categoría única en el DataFrame
    for categoria in df['CATEGORIA'].unique():
        # Filtrar el DataFrame para obtener solo las filas de esta categoría
        df_categoria = df[df['CATEGORIA'] == categoria]
        
        # Iterar sobre cada mes
        for mes_index, mes in enumerate(meses):
            # Inicializar los contadores para ingresos, egresos y permanencias
            ingresos = 0
            egresos = 0
            permanencias = 0

            # Obtener la serie de valores para el mes actual y el mes anterior
            valores_mes_actual = df_categoria[mes].values

            if mes_index == 0:
                # Contar los ingresos iniciales en el primer mes
                ingresos = sum(1 for valor in valores_mes_actual if valor in presente or isinstance(valor, (int, float)))
            else:
                mes_anterior = meses[mes_index - 1]
                valores_mes_anterior = df_categoria[mes_anterior].values
                
                for i, valor_actual in enumerate(valores_mes_actual):
                    valor_anterior = valores_mes_anterior[i] if i < len(valores_mes_anterior) else None

                    # Determinar permanencias
                    if valor_actual in presente or isinstance(valor_actual, (int, float)):
                        if valor_anterior in presente or isinstance(valor_anterior, (int, float)):
                            permanencias += 1
                        else:
                            ingresos += 1
                    elif valor_anterior in presente or isinstance(valor_anterior, (int, float)):
                        egresos += 1

            # Agregar resultados al DataFrame de resultados
            resultados = pd.concat([resultados, pd.DataFrame({
                'CATEGORIA': [categoria],
                'MES': [mes],
                'INGRESOS': [ingresos],
                'EGRESOS': [egresos],
                'PERMANENCIAS': [permanencias],
                'TOTAL ALUMNOS': [ingresos + permanencias -egresos]
            })], ignore_index=True)
    return resultados

In [227]:
def calcular_porcentajes(resultados):
    resultados['% INGRESOS'] = resultados.apply(
        lambda row: round((row['INGRESOS'] / row['TOTAL ALUMNOS']) * 100, 2) if row['TOTAL ALUMNOS'] != 0 else 0, axis=1
    )
    resultados['% EGRESOS'] = resultados.apply(
        lambda row: round((row['EGRESOS'] / row['TOTAL ALUMNOS']) * 100, 2) if row['TOTAL ALUMNOS'] != 0 else 0, axis=1
    )
    
    # Calcular % PERMANENCIAS en base al TOTAL ALUMNOS ANTERIOR
    resultados['% PERMANENCIAS'] = resultados.apply(
        lambda row: round((row['PERMANENCIAS'] / row['TOTAL ALUMNOS ANTERIOR']) * 100, 2) 
        if row['TOTAL ALUMNOS ANTERIOR'] != 0 else 0, axis=1
    )

    return resultados


In [228]:
def calcular_porcentajes(resultados):
    resultados['% INGRESOS'] = resultados.apply(
        lambda row: round((row['INGRESOS'] / row['TOTAL ALUMNOS']) * 100, 2) if row['TOTAL ALUMNOS'] != 0 else 0, axis=1
    )
    resultados['% EGRESOS'] = resultados.apply(
        lambda row: round((row['EGRESOS'] / row['TOTAL ALUMNOS']) * 100, 2) if row['TOTAL ALUMNOS'] != 0 else 0, axis=1
    )
    resultados['% PERMANENCIAS'] = resultados.apply(
        lambda row: (100 - row['% EGRESOS']) if row['TOTAL ALUMNOS'] != 0 else 0, axis=1)
    return resultados

### Voley

In [229]:
iep_voley = ingresos_egresos_permanencias(pagos_por_cat_voley)
resultados_v=calcular_porcentajes(iep_voley)
resultados_v

,CATEGORIA,MES,INGRESOS,EGRESOS,PERMANENCIAS,TOTAL ALUMNOS,% INGRESOS,% EGRESOS,% PERMANENCIAS
0,Recreativo,FEB,0,0,0,0.0,0.00,0.00,0.00
1,Recreativo,MAR,0,0,0,0.0,0.00,0.00,0.00
2,Recreativo,ABR,0,0,0,0.0,0.00,0.00,0.00
3,Recreativo,MAY,0,0,0,0.0,0.00,0.00,0.00
4,Recreativo,JUN,0,0,0,0.0,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...
67,Primera,JUN,5,0,30,35.0,14.29,0.00,100.00
68,Primera,JUL,0,8,27,19.0,0.00,42.11,57.89
69,Primera,AGO,9,8,19,20.0,45.00,40.00,60.00
70,Primera,SEPT,2,2,26,26.0,7.69,7.69,92.31


In [230]:
# Ruta a la carpeta PICKLES (ajusta la ruta si es necesario)
ruta_pickles = os.path.join("..", "PICKLES")

# Asegurarse de que la carpeta PICKLES existe
if not os.path.exists(ruta_pickles):
    os.makedirs(ruta_pickles)

In [231]:
# Ruta completa al archivo pickle
ruta_completa = os.path.join(ruta_pickles, "porcentajes_voley.pkl")
resultados_v.to_pickle(ruta_completa)

### Hand

In [232]:
iep_hand = ingresos_egresos_permanencias(pagos_por_cat_hand)
resultados_h=calcular_porcentajes(iep_hand)
resultados_h

,CATEGORIA,MES,INGRESOS,EGRESOS,PERMANENCIAS,TOTAL ALUMNOS,% INGRESOS,% EGRESOS,% PERMANENCIAS
0,Mini,FEB,5,0,0,5.0,100.00,0.00,100.00
1,Mini,MAR,8,0,5,13.0,61.54,0.00,100.00
2,Mini,ABR,3,0,13,16.0,18.75,0.00,100.00
3,Mini,MAY,5,2,14,17.0,29.41,11.76,88.24
4,Mini,JUN,2,1,18,19.0,10.53,5.26,94.74
5,Mini,JUL,0,3,17,14.0,0.00,21.43,78.57
6,Mini,AGO,3,0,17,20.0,15.00,0.00,100.00
7,Mini,SEPT,2,0,20,22.0,9.09,0.00,100.00
8,Mini,OCT,1,0,22,23.0,4.35,0.00,100.00
9,Infantil,FEB,8,0,0,8.0,100.00,0.00,100.00


In [233]:
# Ruta completa al archivo pickle
ruta_completa = os.path.join(ruta_pickles, "porcentajes_hand.pkl")
resultados_h.to_pickle(ruta_completa)

In [234]:
resultados_h

,CATEGORIA,MES,INGRESOS,EGRESOS,PERMANENCIAS,TOTAL ALUMNOS,% INGRESOS,% EGRESOS,% PERMANENCIAS
0,Mini,FEB,5,0,0,5.0,100.00,0.00,100.00
1,Mini,MAR,8,0,5,13.0,61.54,0.00,100.00
2,Mini,ABR,3,0,13,16.0,18.75,0.00,100.00
3,Mini,MAY,5,2,14,17.0,29.41,11.76,88.24
4,Mini,JUN,2,1,18,19.0,10.53,5.26,94.74
5,Mini,JUL,0,3,17,14.0,0.00,21.43,78.57
6,Mini,AGO,3,0,17,20.0,15.00,0.00,100.00
7,Mini,SEPT,2,0,20,22.0,9.09,0.00,100.00
8,Mini,OCT,1,0,22,23.0,4.35,0.00,100.00
9,Infantil,FEB,8,0,0,8.0,100.00,0.00,100.00


In [235]:
resultados_h.groupby('MES')[['TOTAL ALUMNOS']].sum().reset_index()

,MES,TOTAL ALUMNOS
0,ABR,135.0
1,AGO,148.0
2,FEB,77.0
3,JUL,131.0
4,JUN,145.0
5,MAR,122.0
6,MAY,138.0
7,OCT,151.0
8,SEPT,151.0


Total de activos por mes: (la variable servira para cuando analice los ingresos mes a mes)

In [236]:
resultados_h["SOCIOS"] = resultados_h["INGRESOS"] + resultados_h["PERMANENCIAS"] - resultados_h["EGRESOS"]
socios_por_mes_h = resultados_h.groupby("MES")[["SOCIOS"]].sum().reset_index()
socios_por_mes_h 

,MES,SOCIOS
0,ABR,135
1,AGO,148
2,FEB,77
3,JUL,131
4,JUN,145
5,MAR,122
6,MAY,138
7,OCT,151
8,SEPT,151


In [237]:
resultados_v["SOCIOS"] = resultados_v["INGRESOS"] + resultados_v["PERMANENCIAS"] - resultados_v["EGRESOS"]
socios_por_mes_v = resultados_v.groupby("MES")[["SOCIOS"]].sum().reset_index()
socios_por_mes_v

,MES,SOCIOS
0,ABR,137
1,AGO,131
2,FEB,103
3,JUL,113
4,JUN,135
5,MAR,121
6,MAY,124
7,OCT,149
8,SEPT,135


In [238]:
socios_por_mes_h = pd.Series([135, 148, 77, 131, 145, 122, 138, 155, 151], index=['ABR', 'AGO', 'FEB', 'JUL', 'JUN', 'MAR', 'MAY', 'OCT', 'SEPT'], name='SOCIOS')
socios_por_mes_v = pd.Series([137, 131, 103, 113, 135, 121, 124, 154, 138], index=['ABR', 'AGO', 'FEB', 'JUL', 'JUN', 'MAR', 'MAY', 'OCT', 'SEPT'], name='SOCIOS')
df_socios = pd.DataFrame({'SOCIOS_HAND': socios_por_mes_h, 'SOCIOS_VOLEY': socios_por_mes_v})
df_socios
# Definir el orden específico de los meses
orden_meses = ['FEB', 'MAR', 'ABR', 'MAY', 'JUN', 'JUL', 'AGO', 'SEPT', 'OCT']

# Convertir el índice a una categoría con un orden específico
df_socios.index = pd.Categorical(df_socios.index, categories=orden_meses, ordered=True)
# Ordenar el DataFrame por el índice (meses) según el orden especificado
df_socios_sorted= df_socios.sort_index()
df_socios_sorted

,SOCIOS_HAND,SOCIOS_VOLEY
FEB,77,103
MAR,122,121
ABR,135,137
MAY,138,124
JUN,145,135
JUL,131,113
AGO,148,131
SEPT,151,138
OCT,155,154


## Total deudas

In [239]:
def suma_numericos(df):
    meses = ['FEB', 'MAR', 'ABR', 'MAY', 'JUN', 'JUL', 'AGO', 'SEPT', 'OCT']
    # Crear un diccionario para almacenar las sumas de cada columna
    suma_por_mes = {}
    
    # Iterar sobre cada columna especificada
    for col in meses:
        suma = 0  # Inicializar suma en 0 para cada columna
        for value in df[col]:  # Iterar sobre cada valor de la columna
            # Comprobar si el valor es numérico
            if isinstance(value, (int, float)):
                suma += value  # Sumar si es numérico
        # Convertimos el diccionario a un DataFrame para visualizarlo
        suma_por_mes[col] = suma
    
    return suma_por_mes

### Deudas por mes.

In [240]:
deuda_mensual_hand = suma_numericos(pagos_por_cat_hand)
sum(deuda_mensual_hand.values())
deuda_mensual_hand

{'FEB': 12500,
 'MAR': 30500,
 'ABR': 70100,
 'MAY': 31500,
 'JUN': 21000,
 'JUL': 48000,
 'AGO': 98250,
 'SEPT': 148500,
 'OCT': 217800}

In [241]:
deuda_mensual_voley = suma_numericos(pagos_por_cat_voley)
sum(deuda_mensual_voley.values())
deuda_mensual_voley

{'FEB': 0,
 'MAR': 0,
 'ABR': 24000,
 'MAY': 26500,
 'JUN': 39000,
 'JUL': 51500,
 'AGO': 68500,
 'SEPT': 243750,
 'OCT': 393750}

In [242]:
# Convertir los diccionarios a DataFrames
df_hand = pd.DataFrame(list(deuda_mensual_hand.items()), columns=['MES', 'DEUDA_HAND'])
df_voley = pd.DataFrame(list(deuda_mensual_voley.items()), columns=['MES', 'DEUDA_VOLEY'])

# Realizar el merge en base a la columna 'MES'
deudas_mes = pd.merge(df_hand, df_voley, on='MES')

In [243]:
deudas_mes

,MES,DEUDA_HAND,DEUDA_VOLEY
0,FEB,12500,0
1,MAR,30500,0
2,ABR,70100,24000
3,MAY,31500,26500
4,JUN,21000,39000
5,JUL,48000,51500
6,AGO,98250,68500
7,SEPT,148500,243750
8,OCT,217800,393750


In [244]:
round(pagos_por_cat_hand.groupby('CATEGORIA')[['DEUDA']].mean().reset_index(), 2)

,CATEGORIA,DEUDA
0,Cadetes,1550.00
1,Infantil,5369.05
2,Juveniles,3927.27
3,Menores,658.54
4,Mini,2844.83
5,Primera,4820.90


## Handball

In [245]:
ingresos_hand = pd.read_pickle(r'C:\Users\emita\Bootcamp\EDA\PICKLES\ingresos_reales.pkl') 

#### Comienzo con un df con estadistica descriptiva simple:

In [246]:
ingresos_hand[['MONTO']].describe()

,MONTO
count,1032.000000
mean,9054.047471
std,7880.408736
min,1000.000000
25%,7200.000000
50%,8700.000000
75%,9775.000000
max,209500.000000


In [247]:
ingresos_tot_h = ingresos_hand['MONTO'].sum()
print("El ingreso total en lo que va del anio es:", ingresos_tot_h)

El ingreso total en lo que va del anio es: 9343776.99


In [248]:
# Filtro cuales de los ingresos son pago de cuota de socios
ingresos_cuotas =  ['Cuota', 'Cuota adeudada', 'Cuota medio mes']

pago_cuotas = ingresos_hand[ingresos_hand['DESCRIPCION'].isin(ingresos_cuotas)]
ingresos_por_cat_h = pago_cuotas.groupby('CATEGORIA')[['MONTO']].sum().reset_index()
ingresos_por_cat_h

,CATEGORIA,MONTO
0,Cadetes,1327665.67
1,Infantiles,1117127.33
2,Juveniles,802160.00
3,Mayores,2392880.00
4,Menores,2041597.83
5,Mini,1272846.16


In [249]:
ingreso_cuotas_total = pago_cuotas['MONTO'].sum()
print("El ingreso total en cuotas de socios en lo que va del anio es:", ingreso_cuotas_total)

El ingreso total en cuotas de socios en lo que va del anio es: 8954276.99


In [250]:
porcentaje_cuotas_socios = (ingreso_cuotas_total/ ingresos_tot_h) * 100
print("De los ingresos que recibe el club en el anio, el porcentaje perteneciente a cuotas de socios es de:", porcentaje_cuotas_socios)

De los ingresos que recibe el club en el anio, el porcentaje perteneciente a cuotas de socios es de: 95.83145016820441


In [251]:
# Otros ingresos, que no son cuotas: ['Cambio divisas ahorros', 'SPONSOR CASINO VIRTUAL', 'Sponsor Perramon', 'Eventos']
ingresos_varios = ingresos_tot_h - ingreso_cuotas_total
print(f"Durante el anio hemos recaudado {ingresos_varios} en otros ingresos")

Durante el anio hemos recaudado 389500.0 en otros ingresos


In [252]:
# Calculamos el porcentaje de cuotas pagadas en retardo 
filtro2 = ingresos_hand.loc[:, 'DESCRIPCION'] =='Cuota adeudada'
cuotas_adeudadas = ingresos_hand[filtro2]
cuotas_adeudadas_por_cat = cuotas_adeudadas.groupby('CATEGORIA')[['MONTO']].sum().reset_index()
cuotas_adeudadas_por_cat

,CATEGORIA,MONTO
0,Cadetes,91420.0
1,Infantiles,82500.0
2,Juveniles,62750.0
3,Mayores,165850.0
4,Menores,117850.0


In [253]:
ingresos_por_cat_h = pd.merge(ingresos_por_cat_h, cuotas_adeudadas_por_cat, on='CATEGORIA', how='inner', suffixes=('_Ingresos_cuotas', '_Cuotas_adeudadas'))

In [254]:
ingresos_por_cat_h['PORCENTAJE_DEUDAS'] = round((ingresos_por_cat_h['MONTO_Cuotas_adeudadas']/ingresos_por_cat_h['MONTO_Ingresos_cuotas'])*100,2)
ingresos_por_cat_h

,CATEGORIA,MONTO_Ingresos_cuotas,MONTO_Cuotas_adeudadas,PORCENTAJE_DEUDAS
0,Cadetes,1327665.67,91420.0,6.89
1,Infantiles,1117127.33,82500.0,7.39
2,Juveniles,802160.00,62750.0,7.82
3,Mayores,2392880.00,165850.0,6.93
4,Menores,2041597.83,117850.0,5.77


#### Fecha de los pagos

In [255]:
len(ingresos_hand)

1032

In [256]:
len(pago_cuotas)

1028

### Porcentaje de personas que pagan la primera quincena

In [257]:
# Filtrar las fechas hasta el día 14
primer_quincena = pago_cuotas[pago_cuotas["FECHA"].dt.day <= 14]

# Contar el número de pagos
num_pagos_primer_quincena = primer_quincena.shape[0]

# Mostrar el resultado
print(f"Número de ingresos durante el AÑO en la primera quincena del mes: {num_pagos_primer_quincena}")

Número de ingresos durante el AÑO en la primera quincena del mes: 654


In [258]:
# Filtrar las fechas hasta el día 14
primer_quincena = pago_cuotas[pago_cuotas.loc[:, "FECHA"].dt.day <= 14].copy()  # Hacer una copia explícita

# Extraer el año y mes de la fecha para agrupar usando .loc
primer_quincena.loc[:, "AÑO_MES"] = primer_quincena["FECHA"].dt.to_period("M")

# Contar el número de pagos por mes
h_primera_quincena = primer_quincena.groupby("AÑO_MES").size()

# Mostrar el resultado
print("Número de personas que pagan en la primera quincena del mes:")
print(h_primera_quincena)

Número de personas que pagan en la primera quincena del mes:
AÑO_MES
2024-02     30
2024-03     80
2024-04     80
2024-05     27
2024-06     80
2024-07     91
2024-08     80
2024-09    105
2024-10     81
Freq: M, dtype: int64


In [259]:
# Filtrar las fechas hasta el día 14
al_mes = pago_cuotas[pago_cuotas.loc[:, "FECHA"].dt.day <= 31].copy()  # Hacer una copia explícita

# Extraer el año y mes de la fecha para agrupar usando .loc
al_mes.loc[:, "AÑO_MES"] = al_mes["FECHA"].dt.to_period("M")

# Contar el número de pagos por mes
h_conteo_por_mes = al_mes.groupby("AÑO_MES").size()

# Mostrar el resultado
print("Número de personas que pagan al mes:")
print(h_conteo_por_mes)

Número de personas que pagan al mes:
AÑO_MES
2024-02     66
2024-03    108
2024-04    115
2024-05    118
2024-06    122
2024-07    124
2024-08    131
2024-09    131
2024-10    113
Freq: M, dtype: int64


In [260]:
# Series de datos
pagos_totales = pd.Series(h_conteo_por_mes, name='Pagos Totales')

pagos_quincena = pd.Series(h_primera_quincena, name='Pagos Primera Quincena')

# Combina ambas series en un DataFrame
df_pagos_h = pd.concat([pagos_totales, pagos_quincena], axis=1)

# Calcula el porcentaje de pagos en la primera quincena
df_pagos_h['Porcentaje Primera Quincena'] = round((df_pagos_h['Pagos Primera Quincena'] / df_pagos_h['Pagos Totales']) * 100,2)

# Mostrar el DataFrame resultante
df_pagos_h

,Pagos Totales,Pagos Primera Quincena,Porcentaje Primera Quincena
AÑO_MES,,,
2024-02,66,30,45.45
2024-03,108,80,74.07
2024-04,115,80,69.57
2024-05,118,27,22.88
2024-06,122,80,65.57
2024-07,124,91,73.39
2024-08,131,80,61.07
2024-09,131,105,80.15
2024-10,113,81,71.68


In [261]:
socios_por_mes_hand = pd.DataFrame(socios_por_mes_h).reset_index()
socios_por_mes_hand.columns=['MES', 'NUMERO DE SOCIOS']

In [262]:
meses_orden = ['FEB', 'MAR', 'ABR', 'MAY', 'JUN', 'JUL', 'AGO', 'SEPT', 'OCT']

# Convertir la columna 'MES' a tipo categórico con el orden especificado
socios_por_mes_hand['MES'] = pd.Categorical(socios_por_mes_hand['MES'], categories=meses_orden, ordered=True)

# Ordenar el DataFrame según la columna 'MES'
df_ordenado_h = socios_por_mes_hand.sort_values('MES')
df_ordenado_h

,MES,NUMERO DE SOCIOS
2,FEB,77
5,MAR,122
0,ABR,135
6,MAY,138
4,JUN,145
3,JUL,131
1,AGO,148
8,SEPT,151
7,OCT,155


In [263]:
socios_por_mes_vol = pd.DataFrame(socios_por_mes_v).reset_index()
socios_por_mes_vol.columns=['MES', 'NUMERO DE SOCIOS']

In [264]:
socios_por_mes_vol['MES'] = pd.Categorical(socios_por_mes_vol['MES'], categories=meses_orden, ordered=True)

# Ordenar el DataFrame según la columna 'MES'
df_ordenado_v = socios_por_mes_vol.sort_values('MES')
df_ordenado_v

,MES,NUMERO DE SOCIOS
2,FEB,103
5,MAR,121
0,ABR,137
6,MAY,124
4,JUN,135
3,JUL,113
1,AGO,131
8,SEPT,138
7,OCT,154


In [265]:
socios_por_mes_tot = pd.merge(df_ordenado_h, df_ordenado_v, on='MES')
socios_por_mes_tot.columns=['MES', 'SOCIOS_HAND', 'SOCIOS_VOL']
socios_por_mes_tot

,MES,SOCIOS_HAND,SOCIOS_VOL
0,FEB,77,103
1,MAR,122,121
2,ABR,135,137
3,MAY,138,124
4,JUN,145,135
5,JUL,131,113
6,AGO,148,131
7,SEPT,151,138
8,OCT,155,154


In [266]:
# Ruta completa al archivo pickle
ruta_completa = os.path.join(ruta_pickles, "socios_por_mes_tot.pkl")
socios_por_mes_tot.to_pickle(ruta_completa)

## Voley

In [267]:
ingresos_voley = pd.read_pickle(r'C:\Users\emita\Bootcamp\EDA\PICKLES\ingresos_v.pkl')

In [268]:
ingresos_voley[['MONTO']].describe()

,MONTO
count,1077.000000
mean,7954.549675
std,2825.525119
min,500.000000
25%,6200.000000
50%,7500.000000
75%,9500.000000
max,58000.000000


Las procedencias ('DESCRIPCIÓN') de los distintos ingresos son: ['Cuota', 'Cuota adeudada', 'Cuota medio mes', 'Seguro', 'Devolucion']

Localizo la descripcion del monto mayor:

In [269]:
ingresos_tot_v = ingresos_voley['MONTO'].sum()
print("El ingreso total en lo que va del anio es:", ingresos_tot_v)

El ingreso total en lo que va del anio es: 8567050.0


In [270]:
ingresos_voley['DESCRIPCION'].unique()
# Dejamos afuera de esta lista de ingresos a: 'Devolucion', 'Seguro' porque no son reales ingresos, sino transacciones que se hacen a traves de las arcas del club.
lista_ingresos_reales_v = ['Cuota', 'Cuota adeudada', 'Cuota medio mes'] 
# A diferencia de lo que sucede con Handball, en Voley los unicos ingresos son a traves de las cuotas de los socios
# Por eso directamente dejo este calculo:
# Filtro cuales de los ingresos son pago de cuota de socios
ingresos_cuotas =  ['Cuota', 'Cuota adeudada', 'Cuota medio mes']

pago_cuotas_v = ingresos_voley[ingresos_voley.loc[:, 'DESCRIPCION'].isin(ingresos_cuotas)]
ingresos_por_cat_v = pago_cuotas_v.groupby('CATEGORIA')[['MONTO']].sum().reset_index()
ingresos_por_cat_v

,CATEGORIA,MONTO
0,Mini,501900.0
1,Primera,1598950.0
2,Recreativo,112000.0
3,Su14,10500.0
4,Sub12,961100.0
5,Sub14,1019550.0
6,Sub16,1599550.0
7,Sub18,2000650.0
8,Sub19,741850.0


In [271]:
ingreso_cuotas_total_v = pago_cuotas_v['MONTO'].sum()
print("El ingreso total en cuotas de socios en lo que va del anio es:", ingreso_cuotas_total_v)

El ingreso total en cuotas de socios en lo que va del anio es: 8546050.0


In [272]:
porcentaje_cuotas_socios_v = (ingreso_cuotas_total_v/ ingresos_tot_v) * 100
print("De los ingresos que recibe el club en el anio, el porcentaje perteneciente a cuotas de socios es de:", porcentaje_cuotas_socios_v)

De los ingresos que recibe el club en el anio, el porcentaje perteneciente a cuotas de socios es de: 99.75487478186774


In [273]:
ingresos_varios_v = ingresos_tot_v - ingreso_cuotas_total_v
print(f"Durante el anio hemos recaudado {ingresos_varios_v} en otros ingresos")

Durante el anio hemos recaudado 21000.0 en otros ingresos


In [274]:
# Calculamos el porcentaje de cuotas pagadas en retardo 

In [275]:
pago_cuotas_v

,FECHA,DESCRIPCION,MONTO,CATEGORIA,MEDIO DE PAGO
0,2024-02-05,Cuota,6200.0,Sub16,Transferencia
1,2024-02-05,Cuota,6200.0,Sub18,Transferencia
2,2024-02-05,Cuota,6200.0,Sub16,Transferencia
3,2024-02-05,Cuota,6200.0,Sub18,Transferencia
4,2024-02-05,Cuota,6200.0,Sub14,Transferencia
...,...,...,...,...,...
1072,2024-10-25,Cuota,11250.0,Mini,Transferencia
1073,2024-10-25,Cuota,11250.0,Sub14,Transferencia
1074,2024-10-15,Cuota,10500.0,Sub16,Transferencia
1075,2024-10-28,Cuota,12500.0,Primera,Transferencia


In [276]:
# Calculamos el porcentaje de cuotas pagadas en retardo 
filtro3 = pago_cuotas_v.loc[:, 'DESCRIPCION'] =='Cuota adeudada'
cuotas_adeudadas_v = pago_cuotas_v[filtro3]
cuotas_adeudadas_por_cat_v = cuotas_adeudadas_v.groupby('CATEGORIA')[['MONTO']].sum().reset_index()
cuotas_adeudadas_por_cat_v

,CATEGORIA,MONTO
0,Mini,64750.0
1,Primera,115100.0
2,Recreativo,10500.0
3,Sub12,98500.0
4,Sub14,67200.0
5,Sub16,151000.0
6,Sub18,103500.0
7,Sub19,54750.0


In [277]:
ingresos_por_cat_v = pd.merge(ingresos_por_cat_v, cuotas_adeudadas_por_cat_v, on='CATEGORIA', how='inner', suffixes=('_Ingresos_cuotas', '_Cuotas_adeudadas'))

In [278]:
ingresos_por_cat_v['PORCENTAJE_DEUDAS'] = round((ingresos_por_cat_v['MONTO_Cuotas_adeudadas']/ingresos_por_cat_v['MONTO_Ingresos_cuotas'])*100,2)
ingresos_por_cat_v

,CATEGORIA,MONTO_Ingresos_cuotas,MONTO_Cuotas_adeudadas,PORCENTAJE_DEUDAS
0,Mini,501900.0,64750.0,12.90
1,Primera,1598950.0,115100.0,7.20
2,Recreativo,112000.0,10500.0,9.38
3,Sub12,961100.0,98500.0,10.25
4,Sub14,1019550.0,67200.0,6.59
5,Sub16,1599550.0,151000.0,9.44
6,Sub18,2000650.0,103500.0,5.17
7,Sub19,741850.0,54750.0,7.38


#### Fecha de los pagos

In [279]:
# Numero de ingresos recibidos durante el AÑO, en total
len(pago_cuotas_v)

1075

In [280]:
# Filtrar las fechas hasta el día 14
v_primer_quincena = pago_cuotas_v[pago_cuotas_v["FECHA"].dt.day <= 14]

# Contar el número de pagos
v_num_pagos_primer_quincena = v_primer_quincena.shape[0]

# Mostrar el resultado
print(f"Número de ingresos durante el AÑO en la primera quincena del mes: {v_num_pagos_primer_quincena}")

Número de ingresos durante el AÑO en la primera quincena del mes: 621


In [281]:
# Filtrar las fechas hasta el día 14
v_primer_quincena = pago_cuotas_v[pago_cuotas_v.loc[:, "FECHA"].dt.day <= 14].copy()  # Hacer una copia explícita

# Extraer el año y mes de la fecha para agrupar usando .loc
v_primer_quincena.loc[:, "AÑO_MES"] = v_primer_quincena["FECHA"].dt.to_period("M")

# Contar el número de pagos por mes
v_pagos_prim_quincena = v_primer_quincena.groupby("AÑO_MES").size()

# Mostrar el resultado
print("Número de personas que pagan en la primera quincena del mes:")
print(v_pagos_prim_quincena)

Número de personas que pagan en la primera quincena del mes:
AÑO_MES
2024-02     39
2024-03     94
2024-04     87
2024-05     33
2024-06     61
2024-07     66
2024-08     59
2024-09    100
2024-10     82
Freq: M, dtype: int64


In [282]:
# Filtrar las fechas hasta el día 14
v_al_mes = pago_cuotas_v[pago_cuotas_v.loc[:, "FECHA"].dt.day <= 31].copy()  # Hacer una copia explícita

# Extraer el año y mes de la fecha para agrupar usando .loc
v_al_mes.loc[:, "AÑO_MES"] = v_al_mes["FECHA"].dt.to_period("M")

# Contar el número de pagos por mes
v_conteo_por_mes = v_al_mes.groupby("AÑO_MES").size()

# Mostrar el resultado
print("Número de personas que pagan al mes:")
print(v_conteo_por_mes)

Número de personas que pagan al mes:
AÑO_MES
2024-02     86
2024-03    115
2024-04    141
2024-05    125
2024-06    114
2024-07    121
2024-08    130
2024-09    120
2024-10    123
Freq: M, dtype: int64


In [283]:
# Series de datos
pagos_totales = pd.Series(v_conteo_por_mes, name='Pagos Totales')

pagos_quincena = pd.Series(v_pagos_prim_quincena, name='Pagos Primera Quincena')

# Combina ambas series en un DataFrame
df_pagos_v = pd.concat([pagos_totales, pagos_quincena], axis=1)

# Calcula el porcentaje de pagos en la primera quincena
df_pagos_v['Porcentaje Primera Quincena'] = round((df_pagos_v['Pagos Primera Quincena'] / df_pagos_v['Pagos Totales']) * 100, 2)

# Mostrar el DataFrame resultante
df_pagos_v

,Pagos Totales,Pagos Primera Quincena,Porcentaje Primera Quincena
AÑO_MES,,,
2024-02,86,39,45.35
2024-03,115,94,81.74
2024-04,141,87,61.70
2024-05,125,33,26.40
2024-06,114,61,53.51
2024-07,121,66,54.55
2024-08,130,59,45.38
2024-09,120,100,83.33
2024-10,123,82,66.67


In [284]:
# Montos recibidos en cuotas al mes
v_al_mes = pago_cuotas_v[pago_cuotas_v.loc[:, "FECHA"].dt.day <= 31].copy()  # Hacer una copia explícita

# Extraer el año y mes de la fecha para agrupar usando .loc
pagos_por_mes_v=v_al_mes.loc[:, "AÑO_MES"] = v_al_mes["FECHA"].dt.to_period("M")
ingresos_cuotas_mensual_v = v_al_mes.groupby('AÑO_MES')[['MONTO']].sum().reset_index()

In [285]:
ingresos_cuotas_mensual_v

,AÑO_MES,MONTO
0,2024-02,572400.0
1,2024-03,704000.0
2,2024-04,900900.0
3,2024-05,947300.0
4,2024-06,864650.0
5,2024-07,920200.0
6,2024-08,1049950.0
7,2024-09,1259200.0
8,2024-10,1327450.0


## Resultados hand y voley 

In [286]:
resultados_h['CATEGORIA'].unique()

array(['Mini', 'Infantil', 'Menores', 'Cadetes', 'Juveniles', 'Primera'],
      dtype=object)

In [287]:
resultados_h

,CATEGORIA,MES,INGRESOS,EGRESOS,PERMANENCIAS,TOTAL ALUMNOS,% INGRESOS,% EGRESOS,% PERMANENCIAS,SOCIOS
0,Mini,FEB,5,0,0,5.0,100.00,0.00,100.00,5
1,Mini,MAR,8,0,5,13.0,61.54,0.00,100.00,13
2,Mini,ABR,3,0,13,16.0,18.75,0.00,100.00,16
3,Mini,MAY,5,2,14,17.0,29.41,11.76,88.24,17
4,Mini,JUN,2,1,18,19.0,10.53,5.26,94.74,19
5,Mini,JUL,0,3,17,14.0,0.00,21.43,78.57,14
6,Mini,AGO,3,0,17,20.0,15.00,0.00,100.00,20
7,Mini,SEPT,2,0,20,22.0,9.09,0.00,100.00,22
8,Mini,OCT,1,0,22,23.0,4.35,0.00,100.00,23
9,Infantil,FEB,8,0,0,8.0,100.00,0.00,100.00,8


In [288]:
categorias_hand = resultados_h

In [289]:
resultados_v['CATEGORIA'].unique()

array(['Recreativo', 'Mini', 'Sub12', 'Sub14', 'Sub16', 'Sub18', 'Sub19',
       'Primera'], dtype=object)

In [290]:
categorias_voley = resultados_v

In [291]:
categorias_voley['CATEGORIA'] = categorias_voley['CATEGORIA'].replace({
    'Sub14 fem': 'Sub14',
    'Sub14 masc': 'Sub14', 
    'Sub16 fem': 'Sub16', 
    'Sub16 masc': 'Sub16', 
    'Sub 18 fem': 'Sub18', 
    'Sub 18 masc': 'Sub18', 
    'Sub 19 fem': 'Sub19', 
    'Sub 19 masc': 'Sub19', 
    'Primera fem': 'Primera',
    'Primera masc': 'Primera'
})

In [292]:
# Ruta completa al archivo pickle
ruta_completa = os.path.join(ruta_pickles, "resultados_h.pkl")
categorias_hand.to_pickle(ruta_completa)

In [293]:
# Ruta completa al archivo pickle
ruta_completa = os.path.join(ruta_pickles, "resultados_v.pkl")
categorias_voley.to_pickle(ruta_completa)